In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName('ddam_project')
         .config('spark.some.config.option','some-value')
         .getOrCreate()
         )

In [2]:
# read file from hdfs and infer schema
df_raw = spark.read.csv("hdfs://masterbig-1.itc.unipi.it:54310/user/student18/Hotel_Reviews.csv", header = True, inferSchema = True)
df_raw.printSchema()

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: string (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: string (nullable = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)



In [3]:
from pyspark.sql.functions import monotonically_increasing_id


In [4]:
df_raw_id = df_raw.withColumn('id', monotonically_increasing_id())
df_raw_id.printSchema()
df_raw_id.take(1)

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: string (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: string (nullable = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)
 |-- id: long (nullable = false)



[Row(Hotel_Address=' s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands', Additional_Number_of_Scoring=194, Review_Date='8/3/2017', Average_Score=7.7, Hotel_Name='Hotel Arena', Reviewer_Nationality=' Russia ', Negative_Review=' I am so angry that i made this post available via all possible sites i use when planing my trips so no one will make the mistake of booking this place I made my booking via booking com We stayed for 6 nights in this hotel from 11 to 17 July Upon arrival we were placed in a small room on the 2nd floor of the hotel It turned out that this was not the room we booked I had specially reserved the 2 level duplex room so that we would have a big windows and high ceilings The room itself was ok if you don t mind the broken window that can not be closed hello rain and a mini fridge that contained some sort of a bio weapon at least i guessed so by the smell of it I intimately asked to change the room and after explaining 2 times that i booked a duplex btw it costs t

## Reviews transformation

In [5]:
def cleanReviews(row):
    if row["Negative_Review"] == "No Negative" and row["Positive_Review"] == "No Positive":
        return "EMPTY"
    else:
        if row["Negative_Review"] == "No Negative":
            return (row["Positive_Review"].replace(" i ", " I "))
        elif row["Positive_Review"] == "No Positive":
            return (row["Negative_Review"].replace(" i ", " I "))
        else:
            return(row["Negative_Review"].replace(" i ", " I ") + ". " + row["Positive_Review"].replace(" i ", " I "))
            #users_ratings.append(row[12])       
        

In [7]:
rdd_reviews = df_raw_id.rdd.map(lambda x: (x['id'], cleanReviews(x)) )

In [8]:
# counting rows with empty reviews
rdd_reviews.filter(lambda x: x[1] == "EMPTY").count()

127

## Hotel City-Nationality Extraction

In [96]:
df_raw_id.select('Hotel_Address').distinct().take(5)

[Row(Hotel_Address='7 rue du G n ral de Larminat 15th arr 75015 Paris France'),
 Row(Hotel_Address='De Boelelaan 2 Zuideramstel 1083 HJ Amsterdam Netherlands'),
 Row(Hotel_Address='Kirchengasse 41 07 Neubau 1070 Vienna Austria'),
 Row(Hotel_Address='Rec Comtal 16 18 Ciutat Vella 08003 Barcelona Spain'),
 Row(Hotel_Address='Via Laietana 17 Ciutat Vella 08003 Barcelona Spain')]

In [12]:
# check na value for lat/lng
df_raw_id.select('lat').rdd.filter(lambda x: x['lng']== 'NA').count()


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 8.0 failed 1 times, most recent failure: Lost task 1.0 in stage 8.0 (TID 12, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/hadoopuser/spark/python/lib/pyspark.zip/pyspark/sql/types.py", line 1484, in __getitem__
    idx = self.__fields__.index(item)
ValueError: 'lng' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hadoopuser/spark/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/home/hadoopuser/spark/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/hadoopuser/spark/python/pyspark/rdd.py", line 2407, in pipeline_fu

In [84]:
import reverse_geocode


In [85]:
coordinates = (-37.81, 144.96), (31.76, 35.21)
reverse_geocode.search(coordinates)

[{'city': 'Melbourne', 'country': 'Australia', 'country_code': 'AU'},
 {'city': 'Jerusalem', 'country': 'Israel', 'country_code': 'IL'}]

In [86]:
coordinates

((-37.81, 144.96), (31.76, 35.21))

In [87]:
coord =[ (43.6753176,10.5408628) ]
reverse_geocode.search(coord)

[{'city': 'Cascina', 'country': 'Italy', 'country_code': 'IT'}]

In [109]:
rdd_coord = (
    raw_id.select('id', 'lat', 'lng').rdd
    .filter(lambda x )
    .map(lambda x: (x['id'], [ [float(x['lat']), float(x['lng'])] ] )) 
    .map(lambda x: (x[0], reverse_geocode.search(x[1])))
    )

In [110]:
rdd_coord.take(2)

[(0, [{'city': 'Amsterdam', 'country': 'Netherlands', 'country_code': 'NL'}]),
 (1, [{'city': 'Amsterdam', 'country': 'Netherlands', 'country_code': 'NL'}])]

In [111]:
df_coord = rdd_coord.toDF(['id','geo'])
df_coord.printSchema()

root
 |-- id: long (nullable = true)
 |-- geo: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)



In [112]:
df_raw_id.printSchema()

root
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: string (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Negative_Review: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Positive_Review: string (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: string (nullable = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)
 |-- id: long (nullable = false)



In [113]:
type(df_coord.id)
type(df_raw_id.id)

pyspark.sql.column.Column

In [114]:
df_join = df_raw_id.join(df_coord, ['id'])

In [115]:
df_join.take(1)

Py4JJavaError: An error occurred while calling o916.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 76.0 failed 1 times, most recent failure: Lost task 1.0 in stage 76.0 (TID 311, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/hadoopuser/spark/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/home/hadoopuser/spark/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/hadoopuser/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "<ipython-input-109-1c4f5dc5242e>", line 3, in <lambda>
ValueError: could not convert string to float: 'NA'

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunne